In [1]:
import sys
sys.path.append("../")
sys.path.append("../train_model")

In [2]:
from datetime import datetime
from functools import partial
from matplotlib.dates import DateFormatter
from ta import add_all_ta_features
from ta.utils import dropna
from utils.common_train_utils import *
from utils.draw_candle_image import *
from utils.evaluate_old_models import *
from attention_model import *
import json
import keras
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pathlib
import shutil
import ta
import ta.momentum
import ta.trend
import tempfile
import tensorflow as tf
import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots
keras.config.enable_unsafe_deserialization()

from draw_image_utils import draw_prediction
from config_dataset_and_model import symbol_mapping

2024-12-13 06:22:56.188619: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-13 06:22:56.283211: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-13 06:22:56.338342: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-13 06:22:56.354542: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-13 06:22:56.412765: I tensorflow/core/platform/cpu_feature_guar

In [5]:
symbol = input("Symbol = ").strip().upper()
dataset_folder_name = symbol_mapping[symbol]["dataset_folder"]
model_name = symbol_mapping[symbol]["model_name"]
symbol_name = symbol_mapping[symbol]["symbol_name"]
is_trading_24h = symbol_mapping[symbol]["is_trading_24h"]
dataset_folder = f"../dataset/{dataset_folder_name}"
index_of_image_input = 3
batch_size = 100
freq_of_date = symbol_mapping[symbol]["freq_of_date"] # B or D

config = read_config(dataset_folder)
candle_type_and_directory_save: dict = config["candle_type_and_directory_save"]
previous_days = config["previous_days"]
next_days = config["next_days"]
days_result = config["days_result"]

In [6]:
function_transform_origin = get_origin_transform_function(
    os.path.join(
        os.path.abspath(f"{dataset_folder}/model_save/{model_name}"),
        "config_and_loss.json"
    )
)
path_of_model = f"{dataset_folder}/model_save/{model_name}/{model_name}.keras"
model = keras.models.load_model(os.path.abspath(path_of_model))


I0000 00:00:1734071000.553289  127186 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734071000.730801  127186 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734071000.730866  127186 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734071000.733052  127186 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734071000.733092  127186 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [7]:
dataset_test = load_dataset_of_each_type_and_combine(f"{dataset_folder}/test", candle_type_and_directory_save)
total_data_test = len(dataset_test)


In [8]:
image_shape = get_image_shape(dataset_test, index_of_image_input)

In [9]:
config_of_continue_learning_folder = read_config(dataset_folder)
file_csv_path = config_of_continue_learning_folder["file_csv"]
df_get_from_api = pd.read_csv(file_csv_path)
df_get_from_api["Date"] = pd.to_datetime(df_get_from_api["Date"])

In [69]:
def date_generator(date_i):
    index_of_date = get_date_index(df_get_from_api, date_i)
    total_days = 30 + 1 + 2 + 3
    all_dates = df_get_from_api[index_of_date:index_of_date + 6]["Date"].dt.date
    total_date_reals = len(all_dates)
    dates_real = all_dates
    if len(all_dates) < total_days:
        all_dates = pd.date_range(start=str(date_i), periods=total_days, freq=freq_of_date)
    return list(all_dates), total_date_reals

In [70]:
dataset_test_1 = dataset_test
dataset_test_2 = dataset_test_1.batch(batch_size)
transform_function = partial(function_transform_origin, days_result)
dataset_test_3 = dataset_test_2.map(transform_function)

In [71]:
model.summary()

Model: "cnn_attention_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ cnn_encoder (CNNEncoder)        │ ?                      │    31,844,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder (Decoder)               │ ?                      │       166,786 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 96,033,799 (366.34 MB)

 Trainable params: 32,011,266 (122.11 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 64,022,533 (244.23 MB)

In [72]:
loss = model.evaluate(dataset_test_3)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 707ms/step - loss: 4.8962 - mae: 1.3998 - mse: 4.8962


In [73]:
loss

[4.896203517913818, 1.3997712135314941, 4.896203517913818]

In [74]:
prediction_for_test_origin = model.predict(dataset_test_3)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 368ms/step


In [75]:
prediction_for_test_origin.shape

(64, 4, 2)

In [76]:
prediction_for_test = prediction_for_test_origin[:, -3:, :]

In [77]:
print(prediction_for_test)

[[[-1.97178125e-02  1.51270032e-02]
  [-1.33672762e+00 -1.62325013e+00]
  [-1.96858048e-02  1.51640475e-02]]

 [[ 4.75925207e-03  7.75700808e-02]
  [ 4.75949049e-03  7.75703788e-02]
  [ 4.74750996e-03  7.75610805e-02]]

 [[ 1.93994924e-01  3.04152578e-01]
  [ 1.93993405e-01  3.04150850e-01]
  [ 1.93986163e-01  3.04145634e-01]]

 [[-6.33438528e-02 -3.10711563e-02]
  [-6.33490086e-02 -3.10756862e-02]
  [-6.33440018e-02 -3.10698152e-02]]

 [[ 8.96399021e-02  1.40786886e-01]
  [ 8.96474719e-02  1.40796065e-01]
  [ 8.96443129e-02  1.40791714e-01]]

 [[ 5.63234210e-01  7.32831240e-01]
  [ 5.63144207e-01  7.32725859e-01]
  [ 5.63171089e-01  7.32755899e-01]]

 [[ 2.08528832e-01  3.05613905e-01]
  [ 2.08529532e-01  3.05615127e-01]
  [ 2.08526343e-01  3.05613220e-01]]

 [[ 2.83020973e-01  2.45762691e-01]
  [ 2.83027589e-01  2.45759621e-01]
  [ 2.83038378e-01  2.45768264e-01]]

 [[-1.67707825e+00 -1.91802990e+00]
  [ 4.45167094e-01  5.76411009e-01]
  [ 4.45190370e-01  5.76438665e-01]]

 [[ 2.0583

In [78]:
len(prediction_for_test)

64

In [79]:
all_dates = unwrap_dataset_at_index(dataset_test, 5, to_numpy=True, map_function=lambda date: str(datetime.fromisoformat(date.decode("utf-8")).date()))
# trend_dataset_test_2 = unwrap_dataset_at_index(unwrap_dataset_at_index(dataset_test_2, 2, batch=True, to_numpy=True), 0)
_4_prices_of_all_days_dataset_test_2_origin = unwrap_dataset_at_index(dataset_test_2, 6, batch=True, to_numpy=True)
image_input_dataset_test_2 = unwrap_dataset_at_index(dataset_test_2, 2, batch=True, to_numpy=True)
real_percent_last_days_result_dataset_test_3 = unwrap_dataset_at_index(dataset_test_3, 1, batch=True, to_numpy=True) 

In [80]:
_4_prices_of_all_days_dataset_test_2_origin.shape

(64, 37, 4)

In [81]:
_4_prices_of_all_days_dataset_test_2 = _4_prices_of_all_days_dataset_test_2_origin[:, -6:, :]

In [86]:
_4_prices_of_all_days_dataset_test_2.shape

(64, 6, 4)

In [83]:
mapping_trend_type = {
    1: "GROUP_BULL",
    -1: "GROUP_BEAR",
    0: "GROUP_SIZEWAY",
}

In [84]:
index = 2
print(f"real: {real_percent_last_days_result_dataset_test_3[index]}")
print(f"prediction: {prediction_for_test[index]}")
type_of_output = "open_and_close"
draw_prediction(
    image_input_dataset_test_2[index], 
    _4_prices_of_all_days_dataset_test_2[index], 
    all_dates[index],
    prediction_for_test[index], 
    show_x_orginal_candle=True,
    show_prediction_candle=True, 
    show_original_candle=True,
    show_close_compare=True,
    show_open_compare=True,
    type_of_output=type_of_output,
    draw_beside=True,
    print_image=True,
    extend_real=True,
    date_generator=date_generator,
    symbol=symbol_name)

real: [[ 1.4180866   0.        ]
 [ 0.91440314  3.2017798 ]
 [ 0.13387097  0.91440314]
 [ 0.         -0.25535262]]
prediction: [[0.19399492 0.30415258]
 [0.1939934  0.30415085]
 [0.19398616 0.30414563]]


ValueError: Length of values (36) does not match length of index (6)

In [56]:
# index = 0
is_print_all = input("Print all? (y/n): ").lower()[0] == "y"
type_of_output = "open_and_close"
if is_print_all:
    group = input("Group to print? (bull/bear/sizeway/all) ").strip().lower()
    is_extend_real = input("Extend real? (y/n): ").strip().lower()[0] == "y"
    days_prediction_to_print=input(f"days_prediction_to_print = (default = {days_result})").strip()
    if not days_prediction_to_print:
        days_prediction_to_print = days_result
    else:
        days_prediction_to_print = int(days_prediction_to_print)
    
    # if "bull" in group:
    #     group = "GROUP_BULL"
    # elif "bear" in group:
    #     group = "GROUP_BEAR"
    # elif "sizeway" in group:
    #     group = "GROUP_SIZEWAY"
    if len(group) == 0 or "all" in group:
        group = "ALL"
        
    folder_name = f"{model_name}"
    # if group is not None:
    #     folder_name += f"/{group}"
    
    if is_extend_real:
        folder_name = f"extend_real_{folder_name}"
    folder_save_image_predict = f"{dataset_folder}/output_prediction_image/{folder_name}_prediction_{days_prediction_to_print}"
    
    os.makedirs(folder_save_image_predict, exist_ok=False)
    
    date_printed = set()
    for index in range(len(_4_prices_of_all_days_dataset_test_2)):
        if index % 10 == 0:
            print(f"---------------- Save: {index}/{len(_4_prices_of_all_days_dataset_test_2)} images")
        print(f"real: {real_percent_last_days_result_dataset_test_3[index]}")
        print(f"prediction: {prediction_for_test[index]}")
        date = all_dates[index]
        if date in date_printed:
            continue
        else:
            date_printed.add(date)
        
        # trend_type = mapping_trend_type[trend_dataset_test_2[index]]
        
        # if group is not None and group != "ALL":
        #     if trend_type != group:
        #         continue
        
        draw_prediction(
            image_input_dataset_test_2[index], 
            _4_prices_of_all_days_dataset_test_2[index], 
            date,
            prediction_for_test[index], 
            show_x_orginal_candle=True,
            show_prediction_candle=True, 
            show_original_candle=True,
            show_close_compare=True,
            show_open_compare=True,
            type_of_output=type_of_output,
            draw_beside=True,
            save_image=f"{folder_save_image_predict}/{date}_predict.png",
            print_image=False,
            extend_real=is_extend_real,
            date_generator=date_generator,
            symbol=symbol_name,
            days_prediction_to_print=days_prediction_to_print,
            print_all_days_for_prediction=False,
            is_trading_24h=is_trading_24h)

---------------- Save: 0/64 images
real: [[ 0.          3.20082   ]
 [-0.83386713 -1.4351993 ]
 [-1.4351993  -1.5524982 ]
 [ 1.9988778   0.        ]]
prediction: [[-0.019687    0.01516423]
 [-0.01971781  0.015127  ]
 [-1.3367276  -1.6232501 ]
 [-0.0196858   0.01516405]]
real: [[1.9301579  2.3908412 ]
 [0.40201843 1.9301579 ]
 [0.23404352 0.40201843]
 [0.36153904 0.23404352]]
prediction: [[0.00475931 0.07756978]
 [0.00475925 0.07757008]
 [0.00475949 0.07757038]
 [0.00474751 0.07756108]]
real: [[ 1.4180866   0.        ]
 [ 0.91440314  3.2017798 ]
 [ 0.13387097  0.91440314]
 [ 0.         -0.25535262]]
prediction: [[0.19396326 0.3041263 ]
 [0.19399492 0.30415258]
 [0.1939934  0.30415085]
 [0.19398616 0.30414563]]
real: [[ 0.88266814  0.        ]
 [ 0.         -1.46319   ]
 [-2.3264463   0.        ]
 [ 0.         -1.0520598 ]]
prediction: [[-0.06335905 -0.03108099]
 [-0.06334385 -0.03107116]
 [-0.06334901 -0.03107569]
 [-0.063344   -0.03106982]]
real: [[-1.6210545  0.       ]
 [ 1.3845805  

ValueError: Length of values (36) does not match length of index (37)

In [ ]:
folder_save_image_predict

'../dataset/btc_with_ema_macd_trend/output_prediction_image/extend_real_channels_to_time_step_attention_ema_macd_trend/ALL_prediction_3'

: 